In [1]:
pip install pandas numpy shapely pyproj geopandas rtree tqdm

Note: you may need to restart the kernel to use updated packages.


In [2]:
# pip install pandas numpy shapely pyproj geopandas rtree tqdm

import zipfile, pandas as pd, numpy as np
from datetime import datetime, timedelta, date
from shapely.geometry import LineString
from shapely.ops import substring
from pyproj import Geod
from tqdm import tqdm

In [6]:
# -----------------------------
# CONFIG — edit these first
# -----------------------------
GTFS_ZIP = "gtfs_puget_sound_consolidated.zip"  # your uploaded file name
GTFS_BASE = ""    # inner folder name in the zip

SERVICE_DATE = "2025-09-10"   # YYYY-MM-DD (weekday inside calendar range)
TIME_STEP_SEC = 30            # sampling step (20–30s looks nice)
TIME_START = "06:00:00"       # optional window start (set to None for full day)
TIME_END   = "10:00:00"       # optional window end   (set to None for full day)

# Keep only these route_types, or set to None for all
# GTFS route_type: 0=Tram/LightRail, 1=Subway, 2=Rail, 3=Bus, 4=Ferry, 12=Monorail (sometimes 11/12 vary by feed)
ROUTE_TYPES_INCLUDE = [0, 2, 3, 4]   # light rail/tram, rail (Sounder), bus, ferry

In [7]:
# Common dtype map for GTFS IDs (force to strings to avoid mixed-type warnings)
ID_DTYPE = {
    "agency_id": str,
    "route_id": str,
    "trip_id": str,
    "service_id": str,
    "shape_id": str,
    "stop_id": str,
    "block_id": str,
}

def _read_csv_from_zip(zf, name, usecols=None, extra_dtype=None):
    # Merge the base ID dtype map with any file-specific overrides
    dtype = dict(ID_DTYPE)
    if extra_dtype:
        dtype.update(extra_dtype)
    return pd.read_csv(
        zf.open(GTFS_BASE + name),
        usecols=usecols,
        dtype=dtype,
        low_memory=False
    )

def service_ids_by_date(cal, cald, the_date: date):
    """Compute active service_ids for a given date using calendar + calendar_dates rules."""
    weekday = the_date.weekday()  # Mon=0 ... Sun=6
    wkcols = ["monday","tuesday","wednesday","thursday","friday","saturday","sunday"]
    wkcol = wkcols[weekday]

    active = cal[
        (cal["start_date"] <= pd.Timestamp(the_date)) &
        (cal["end_date"]   >= pd.Timestamp(the_date)) &
        (cal[wkcol] == 1)
    ]["service_id"].astype(str).tolist()
    active = set(active)

    if cald is not None and len(cald):
        sel = cald[cald["date"] == pd.Timestamp(the_date)]
        for _, r in sel.iterrows():
            sid = str(r["service_id"])
            if r["exception_type"] == 1:   # added service
                active.add(sid)
            elif r["exception_type"] == 2: # removed service
                active.discard(sid)

    return active

# Load GTFS safely with explicit dtypes
with zipfile.ZipFile(GTFS_ZIP) as zf:
    routes = _read_csv_from_zip(
        zf, "routes.txt",
        usecols=["route_id","route_type","agency_id"],
        extra_dtype={"route_type": "Int64"}  # allow NA int
    )
    trips  = _read_csv_from_zip(
        zf, "trips.txt",
        usecols=["route_id","trip_id","service_id","shape_id","block_id"]
    )
    # times as strings to preserve HH:MM:SS possibly >24h
    stop_times = _read_csv_from_zip(
        zf, "stop_times.txt",
        extra_dtype={
            "arrival_time": str,
            "departure_time": str,
            "stop_sequence": "Int64",
            "shape_dist_traveled": "float64"
        }
    )
    # shapes lat/lon strictly numeric; ids as strings
    shapes = _read_csv_from_zip(
        zf, "shapes.txt",
        extra_dtype={
            "shape_pt_lat": "float64",
            "shape_pt_lon": "float64",
            "shape_pt_sequence": "Int64"
        }
    )
    cal = _read_csv_from_zip(zf, "calendar.txt")
    cald = _read_csv_from_zip(zf, "calendar_dates.txt")


In [8]:
# Parse calendar date fields
for col in ("start_date", "end_date"):
    cal[col] = pd.to_datetime(cal[col], format="%Y%m%d")
cald["date"] = pd.to_datetime(cald["date"], format="%Y%m%d")

# Pick service date and compute active services
svc_dt = pd.to_datetime(SERVICE_DATE).date()
active_sids = service_ids_by_date(cal, cald, svc_dt)


In [9]:
# Filter routes by route_type if requested
if ROUTE_TYPES_INCLUDE is not None and "route_type" in routes.columns:
    routes = routes[routes["route_type"].isin(ROUTE_TYPES_INCLUDE)]


In [10]:
# Join trips with routes and keep only active services
tr = trips.merge(routes[["route_id","route_type","agency_id"]], on="route_id", how="inner")
tr = tr[tr["service_id"].astype(str).isin(active_sids)].copy()

In [11]:
# Keep only needed stop_times columns
keep_cols = ["trip_id","arrival_time","departure_time","stop_sequence"]
has_sdt = "shape_dist_traveled" in stop_times.columns
if has_sdt:
    keep_cols.append("shape_dist_traveled")
stop_times = stop_times[keep_cols].copy()
stop_times["stop_sequence"] = stop_times["stop_sequence"].astype("Int64")
stop_times = stop_times.sort_values(["trip_id","stop_sequence"])

In [12]:
# Sort shapes and build LineStrings
shapes = shapes.sort_values(["shape_id","shape_pt_sequence"])
shape_lines = {}
for sid, grp in shapes.groupby("shape_id", sort=False):
    coords = grp[["shape_pt_lon","shape_pt_lat"]].to_numpy()
    # (x, y) = (lon, lat)
    shape_lines[str(sid)] = LineString(coords)

In [13]:
# Precompute geodesic length for each shape (meters)
geod = Geod(ellps="WGS84")
def geod_length_m(line: LineString) -> float:
    lons, lats = line.xy
    if len(lons) < 2:
        return 0.0
    return sum(geod.line_length([lons[i], lons[i+1]], [lats[i], lats[i+1]]) for i in range(len(lons)-1))

shape_lengths = {sid: geod_length_m(ln) for sid, ln in shape_lines.items()}

In [14]:
# Time parsing helpers
def to_timedelta(hms: str):
    # Robust to strings like "25:10:00"
    h, m, s = [int(x) for x in hms.split(":")]
    return timedelta(hours=h, minutes=m, seconds=s)

window_start = to_timedelta(TIME_START) if TIME_START else None
window_end   = to_timedelta(TIME_END)   if TIME_END else None

def interpolate_trip(trip_id: str, shape_id: str, times_df: pd.DataFrame) -> pd.DataFrame:
    """Interpolate positions for a single trip into (timestamp, lon, lat) using integer seconds."""
    shape_id = str(shape_id)
    if shape_id not in shape_lines:
        return pd.DataFrame()
    line = shape_lines[shape_id]
    if line.length == 0:
        return pd.DataFrame()

    df = times_df.sort_values("stop_sequence").copy()

    # Build time column (prefer departure_time), keep only valid HH:MM:SS rows
    use_dep = df["departure_time"].notna()
    df["t_str"] = df["departure_time"].where(use_dep, df["arrival_time"])
    df = df[df["t_str"].notna()].copy()
    mask = df["t_str"].astype(str).str.match(r"^\d{1,2}:\d{2}:\d{2}$")
    df = df[mask].copy()
    if df.empty:
        return pd.DataFrame()

    # Convert "HH:MM:SS" -> integer seconds (supports HH >= 24)
    def hms_to_sec(s: str) -> int:
        h, m, sec = map(int, s.split(":"))
        return h*3600 + m*60 + sec

    df["t_sec"] = df["t_str"].astype(str).apply(hms_to_sec)

    # Fractions along the shape
    if has_sdt and df.get("shape_dist_traveled") is not None:
        L_m = shape_lengths.get(shape_id, None)
        if L_m and L_m > 0 and df["shape_dist_traveled"].notna().any():
            df["f"] = df["shape_dist_traveled"].astype(float) / L_m
            df["f"] = df["f"].clip(0, 1)
        else:
            n = len(df)
            df["f"] = np.linspace(0, 1, n)
    else:
        n = len(df)
        df["f"] = np.linspace(0, 1, n)

    # Determine interpolation window in seconds
    t0_sec = int(df["t_sec"].iloc[0])
    tN_sec = int(df["t_sec"].iloc[-1])

    def to_sec_or_none(hms):
        if hms is None: return None
        h, m, s = map(int, hms.split(":"))
        return h*3600 + m*60 + s

    ws_sec = to_sec_or_none(TIME_START)
    we_sec = to_sec_or_none(TIME_END)

    start_sec = ws_sec if ws_sec is not None else t0_sec
    end_sec   = we_sec if we_sec is not None else tN_sec
    # Clamp to trip bounds
    start_sec = max(t0_sec, start_sec)
    end_sec   = min(tN_sec, end_sec)
    if end_sec <= start_sec:
        return pd.DataFrame()

    # Ensure monotonic times (rare, but guard against duplicates)
    df = df.sort_values("t_sec").drop_duplicates(subset=["t_sec"])
    T_sec = df["t_sec"].to_numpy()
    F = df["f"].to_numpy()

    # Sample timestamps (integer seconds)
    ts_sec = np.arange(start_sec, end_sec + 1, TIME_STEP_SEC, dtype=int)

    rows = []
    for tt in ts_sec:
        # find segment index
        j = np.searchsorted(T_sec, tt) - 1
        if j < 0:
            j = 0
        if j >= len(T_sec) - 1:
            j = len(T_sec) - 2
        t1, t2 = T_sec[j], T_sec[j+1]
        f1, f2 = F[j], F[j+1]
        if t2 == t1:
            f = float(f2)
        else:
            f = float(f1 + (f2 - f1) * ((tt - t1) / (t2 - t1)))
        f = max(0.0, min(1.0, f))
        sub = substring(line, 0, f, normalized=True)
        x, y = sub.coords[-1]
        rows.append((tt, x, y))

    out = pd.DataFrame(rows, columns=["t_sec", "lon", "lat"])
    out["trip_id"] = str(trip_id)
    out["shape_id"] = shape_id
    return out


In [15]:
# Build trip→shape metadata
tr_sel = tr.dropna(subset=["trip_id","shape_id"]).copy()
tr_sel["trip_id"] = tr_sel["trip_id"].astype(str)
tr_sel["shape_id"] = tr_sel["shape_id"].astype(str)
trip_meta = tr_sel.set_index("trip_id")[["route_id","route_type","agency_id","shape_id"]]


In [16]:
# Interpolate all trips
points = []
for trip_id, g in tqdm(stop_times.groupby("trip_id"), desc="Interpolating trips"):
    tid = str(trip_id)
    if tid not in trip_meta.index:
        continue
    meta = trip_meta.loc[tid]
    shp = meta["shape_id"]
    dfp = interpolate_trip(tid, shp, g)
    if len(dfp):
        dfp["route_id"]   = meta["route_id"]
        dfp["route_type"] = meta["route_type"]
        dfp["agency_id"]  = meta["agency_id"]
        points.append(dfp)

if not points:
    raise SystemExit("No points generated. Check SERVICE_DATE and time window against calendar.")

pts = pd.concat(points, ignore_index=True)

Interpolating trips: 100%|██████████| 129683/129683 [04:42<00:00, 459.76it/s] 


In [17]:
# Compose absolute datetime for QGIS Time Manager
service_day = pd.Timestamp(SERVICE_DATE)
# if interpolate_trip outputs 't_sec' instead of 't'
pts["datetime"] = service_day + pd.to_timedelta(pts["t_sec"], unit="s")

# Save CSV
OUT_CSV = "seattle_transit_points.csv"
pts.to_csv(OUT_CSV, index=False)
print(f"Wrote {len(pts):,} points to {OUT_CSV}")
print(pts.head())

Wrote 419,118 points to seattle_transit_points.csv
   t_sec         lon        lat  trip_id    shape_id route_id  route_type  \
0  32400 -122.512384  47.165187  1006020  shp-202-04      202           3   
1  32430 -122.509473  47.165168  1006020  shp-202-04      202           3   
2  32460 -122.507542  47.166495  1006020  shp-202-04      202           3   
3  32490 -122.506571  47.168621  1006020  shp-202-04      202           3   
4  32520 -122.505770  47.170792  1006020  shp-202-04      202           3   

  agency_id            datetime  
0         3 2025-09-10 09:00:00  
1         3 2025-09-10 09:00:30  
2         3 2025-09-10 09:01:00  
3         3 2025-09-10 09:01:30  
4         3 2025-09-10 09:02:00  


In [18]:
import pandas as pd

df = pd.read_csv("seattle_transit_points.csv", parse_dates=["datetime"])
# Each point will remain visible for 2 minutes
df["end_datetime"] = df["datetime"] + pd.to_timedelta(2, unit="m")
df.to_csv("seattle_transit_points_with_end.csv", index=False)


C:\Users\lette\AppData\Local\Temp\ipykernel_449672\2359796962.py:3: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("seattle_transit_points.csv", parse_dates=["datetime"])


In [19]:
import pandas as pd

# Input + output file paths
IN_CSV = "seattle_transit_points_with_end.csv"            # or "seattle_transit_points_with_end.csv"
OUT_CSV = "seattle_transit_points_moving.csv"

# Load data
df = pd.read_csv(IN_CSV, parse_dates=["datetime"])

# Find trips that never change position
static_trips = (
    df.groupby("trip_id")
      .agg(lon_unique=("lon", "nunique"),
           lat_unique=("lat", "nunique"))
)
static_trips = static_trips[
    (static_trips["lon_unique"] == 1) & (static_trips["lat_unique"] == 1)
]

print(f"Found {len(static_trips)} static trips")

# Filter them out
df_clean = df[~df["trip_id"].isin(static_trips.index)]

# Save
df_clean.to_csv(OUT_CSV, index=False)
print(f"Saved cleaned file with {len(df_clean):,} points → {OUT_CSV}")


C:\Users\lette\AppData\Local\Temp\ipykernel_449672\79139674.py:8: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(IN_CSV, parse_dates=["datetime"])


Found 195 static trips
Saved cleaned file with 412,782 points → seattle_transit_points_moving.csv


##Charting